In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

 99% 1.05G/1.06G [00:08<00:00, 241MB/s]
100% 1.06G/1.06G [00:08<00:00, 134MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import seaborn
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,BatchNormalization,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
batch_size = 32

# this is the augmentatin configuration we will use for traning
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range=0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# batches of augmentation of data
train_data = train_datagen.flow_from_directory(
    "/content/train",
    target_size = (256,256), # All Image Will Resize 256x256
    batch_size = batch_size,
    class_mode = "binary"
)

validation_data = test_datagen.flow_from_directory(
    "/content/test",
    target_size = (256,256), # All Image Will Resize 256x256
    batch_size = batch_size,
    class_mode = "binary"
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [13]:
## Create LeNet Arcticture with some modification

model = Sequential()

# in this layer they add tanh activation function  i m using relu and also padding
model.add(Conv2D(6,kernel_size = (5,5),padding="same",activation="relu",input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2)) # i am using max pooling they use average pooling

model.add(Conv2D(16,kernel_size = (5,5),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

# add flatten layer to convet iin to 1D array
model.add(Flatten())

# now create ANN layers Fully Connected
model.add(Dense(120,activation="relu"))
model.add(Dense(84,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 256, 256, 6)       456       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 128, 128, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 16)      2416      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 64, 64, 16)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 65536)             0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [15]:
model.compile(loss = tensorflow.keras.losses.binary_crossentropy,
              optimizer="Adam",
              metrics = ["accuracy"]
              )

In [17]:
model.fit_generator(
    train_data,
    steps_per_epoch = 20000//batch_size,
    epochs = 5,
    validation_data = validation_data,
    validation_steps = 5000//batch_size
)

Epoch 1/5
625/625 [==============================] - 312s 500ms/step - loss: 0.5995 - accuracy: 0.6805 - val_loss: 0.5893 - val_accuracy: 0.6862
Epoch 2/5
625/625 [==============================] - 316s 505ms/step - loss: 0.5474 - accuracy: 0.7208 - val_loss: 0.5411 - val_accuracy: 0.7000
Epoch 3/5
625/625 [==============================] - 311s 498ms/step - loss: 0.4987 - accuracy: 0.7568 - val_loss: 0.4918 - val_accuracy: 0.7763
Epoch 4/5
625/625 [==============================] - 313s 501ms/step - loss: 0.4697 - accuracy: 0.7786 - val_loss: 0.4587 - val_accuracy: 0.7887
Epoch 5/5
625/625 [==============================] - 310s 496ms/step - loss: 0.4421 - accuracy: 0.7908 - val_loss: 0.4749 - val_accuracy: 0.7800
